# Import

In [43]:
import numpy as np
import pandas as pd

# Data preporation

In [44]:
# Load previous day data
file_name = 'release/covid19-russia-cases.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
4833,2020-05-30,Челябинская область,Chelyabinsk region,74.0,292.0,2.0,152.0,2878.0,17.0,1258.0
4834,2020-05-30,Чеченская Республика,Chechen Republic,95.0,17.0,0.0,38.0,1209.0,13.0,825.0
4835,2020-05-30,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,5.0,0.0,3.0,78.0,1.0,53.0
4836,2020-05-30,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,30.0,0.0,55.0,2310.0,9.0,953.0
4837,2020-05-30,Ярославская область,Yaroslavl region,76.0,79.0,0.0,53.0,3055.0,14.0,751.0


In [45]:
# Read update data
# Result of data_parsing.ipynb
file_name = 'covid19-russia-cases-upd.csv'
upd_df = pd.read_csv(file_name)
upd_df.sort_values('Confirmed', ascending=False).head()

,Region/City,Confirmed,Date,Deaths,Recovered,Day-Confirmed,Day-Deaths,Day-Recovered
29,Москва,180791.0,2020-05-31,2477.0,80179.0,2595.0,69.0,1855.0
30,Московская область,38995.0,2020-05-31,449.0,8091.0,757.0,25.0,371.0
64,Санкт-Петербург,15949.0,2020-05-31,204.0,5680.0,369.0,13.0,307.0
33,Нижегородская область,9834.0,2020-05-31,98.0,3693.0,301.0,5.0,56.0
67,Свердловская область,5449.0,2020-05-31,27.0,2173.0,265.0,1.0,31.0


In [46]:
# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b') 
upd_df['Region/City'] = upd_df['Region/City'].astype('str').str.strip('\u200b')

# Updating

In [47]:
def upd(row):
    reg = row['Region/City']
    
    row['Confirmed'] = rus_df[rus_df['Region/City'] == reg]['Confirmed'].max() + row['Day-Confirmed']
    row['Deaths'] = rus_df[rus_df['Region/City'] == reg]['Deaths'].max() + row['Day-Deaths']
    row['Recovered'] = rus_df[rus_df['Region/City'] == reg]['Recovered'].max() + row['Day-Recovered']
    
    row['Confirmed'] = row['Day-Confirmed'] if np.isnan(row['Confirmed']) else row['Confirmed']
    row['Deaths'] = row['Day-Deaths'] if np.isnan(row['Deaths']) else row['Deaths']
    row['Recovered'] = row['Day-Recovered'] if np.isnan(row['Recovered']) else row['Recovered']
    
    return row.drop('Region/City')

# If cumulative values unknown
# upd_df = upd_df.groupby('Region/City').apply(lambda df: upd(df.iloc[0])).reset_index()

In [48]:
# Date to datetime
upd_df['Date'] = pd.to_datetime(upd_df['Date'])
rus_df['Date'] = pd.to_datetime(rus_df['Date'])

In [49]:
# Region names translation
# from googletrans import Translator
# translator = Translator()

# def translate(rus):
#     eng = translator.translate(rus, src='ru', dest='en')
#     return eng.text

# upd_df['Region/City-Eng'] = upd_df['Region/City'].apply(translate)

In [50]:
# Add region ID and Eng columns
regions_df = pd.read_csv('release/regions-info.csv')

right = regions_df[['Region_ID', 'Region', 'Region_eng']]
upd_df = upd_df.merge(right, left_on='Region/City', right_on='Region', how='left')
upd_df['Region/City-Eng'] = upd_df.Region_eng
upd_df = upd_df[['Date', 'Region/City', 'Region/City-Eng', 'Region_ID', 'Day-Confirmed', 'Day-Deaths', 'Day-Recovered', 'Confirmed', 'Deaths', 'Recovered']]

In [51]:
rus_df = pd.concat([rus_df, upd_df]).reset_index(drop=True)
rus_df.tail(10)

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
4912,2020-05-31,Тульская область,Tula region,71.0,104.0,0.0,24.0,3855.0,36.0,1803.0
4913,2020-05-31,Тюменская область,Tyumen region,72.0,30.0,0.0,22.0,1638.0,8.0,812.0
4914,2020-05-31,Удмуртская Республика,Udmurt republic,18.0,23.0,0.0,1.0,652.0,12.0,411.0
4915,2020-05-31,Ульяновская область,Ulyanovsk region,73.0,97.0,0.0,13.0,2466.0,8.0,713.0
4916,2020-05-31,Хабаровский край,Khabarovsk region,27.0,63.0,0.0,44.0,2081.0,11.0,1002.0
4917,2020-05-31,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,136.0,0.0,30.0,2436.0,18.0,782.0
4918,2020-05-31,Чеченская Республика,Chechen Republic,95.0,18.0,0.0,3.0,1227.0,13.0,828.0
4919,2020-05-31,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,2.0,0.0,1.0,80.0,1.0,54.0
4920,2020-05-31,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,65.0,0.0,22.0,2375.0,9.0,975.0
4921,2020-05-31,Ярославская область,Yaroslavl region,76.0,77.0,0.0,5.0,3132.0,14.0,756.0


In [52]:
# Negative values fixing
# Negative values can appear if some cases was removed from statistic
def negval_fix(reg):
    last = reg.iloc[-1]
    diff_c = last['Day-Confirmed']*(-1) if last['Day-Confirmed'] < 0 else 0
    diff_d = last['Day-Deaths']*(-1) if last['Day-Deaths'] < 0 else 0
    diff_r = last['Day-Recovered']*(-1) if last['Day-Recovered'] < 0 else 0
    
    last['Day-Confirmed'] += diff_c
    last['Day-Deaths']    += diff_d
    last['Day-Recovered'] += diff_r
    reg.iloc[-1] = last
    
    if diff_c + diff_d + diff_r > 0:
        print(f"Conf {diff_c} Death {diff_d} Rec {diff_r} {last['Region/City']}")
        prev = reg.iloc[-2]
        prev['Confirmed'] -= diff_c
        prev['Deaths']    -= diff_d
        prev['Recovered'] -= diff_r
        reg.iloc[-2] = prev
    
    return reg
    
rus_df = rus_df.groupby('Region/City').apply(negval_fix).reset_index(drop=True)

In [53]:
# Manual fixes if needed
# rus_df.loc[321, 'Confirmed'] = 5
# rus_df.loc[359, 'Day-Confirmed'] = 0


# rus_df.loc[3775, 'Deaths'] = 7
# rus_df.loc[3775, 'Day-Deaths'] = 0

# Data checking

In [54]:
rus_sum = rus_df.loc[rus_df['Region/City'] != 'Diamond Princess']

print('Russia Confirmed:', rus_sum.groupby('Region/City')['Confirmed'].max().sum(),
      rus_sum.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', rus_sum.groupby('Region/City')['Deaths'].max().sum(),
      rus_sum.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', rus_sum.groupby('Region/City')['Recovered'].max().sum(),
      rus_sum.groupby('Region/City')['Day-Recovered'].sum().sum())

rus_regs = rus_sum.groupby('Region/City')['Confirmed'].max().reset_index()
rus_regs['Confirmed-ByDay'] = rus_sum.groupby('Region/City')['Day-Confirmed'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Confirmed'] != rus_regs['Confirmed-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Deaths'].max().reset_index()
rus_regs['Deaths-ByDay'] = rus_sum.groupby('Region/City')['Day-Deaths'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Deaths'] != rus_regs['Deaths-ByDay']])

# rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 405926.0 405926.0
Russia Deaths: 4698.0 4698.0
Russia Recovered: 172046.0 172046.0
Empty DataFrame
Columns: [Region/City, Confirmed, Confirmed-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Recovered, Recovered-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Deaths, Deaths-ByDay]
Index: []


# Data saving

In [55]:
file_name = 'release/covid19-russia-cases.csv'
rus_df.to_csv(file_name, index=False)